# Requirements

In [ ]:
%pip install -r requirements.txt

In [1]:
!rm -f /data2/AAG/Audio_Declip/logs/*.log

# Codes For Background Run

In [ ]:
# nohup python train_data_gen.py --audio_dir "/data/AAG/MTech_Project_Data/speech_data_filter"  --cnt 2500  --train_dir "train_data" --test_dir "test_data" --output_path "pkl_data"  --target_fs_values 16000 --clipping_thresholds 0.1 0.2 --time_clip 1 --win_len 500  --win_shift 125 --delta 300 --s_ratio 0.9 > logs/data_gen_log.txt 2>&1 &

In [ ]:
# !python segregate.py --audio_dir "/data2/AAG/Ausio_Declip/MTech_Project_Data/speech_data_filter" \
#     --cnt 10 --train_dir "train_data" --test_dir "test_data" --s_ratio 0.9 

In [23]:
# nohup python train_data_gen.py --audio_dir "/data2/AAG/Audio_Declip/train_data" --output_path "pkl_data" --target_fs_values 16000 --clipping_thresholds 0.2 0.4 0.6 0.8 --time_clip 1 --win_len 500 --win_shift 125 --delta 300 --n_files 20 > logs/train_gen.log 2>&1 &

In [ ]:
# nohup python training.py --pkl_path pkl_data/training_data.pkl --epochs 50 --batch_size 128 --save_path saved_models --plot_path loss_plots --checkpoint_freq 50 --val_split 0.2 --resume --val --dft_size 1000 --max_sparsity 500 --mask_size 500 > logs/training_log.log 2>&1 &

# Main deployment

In [ ]:
!python evaluate.py --model_path "saved_models/final/complex_dft_unet_final.pth" \
    --test_audio_dir "custom_sound" \
    --output_dir "custom_sound_output" \
    --target_fs_values 16000 \
    --clipping_thresholds 0.2 \
    --time_clip 1 \
    --factor 0.8 \
    --eval_mode 1 \
    --dynamic 0 \
    --save 0 \
    --delta 300 \
    --c_win 500 \
    --r_mode 1 \
    --verbose 1

Processing files:   0%|                                   | 0/1 [00:00<?, ?it/s]
Processing: 61-70968-0000.wav (fs=16000, threshold=0.2, duration=1, winlen=500)
Generating clipped signal...
Processing files: 100%|███████████████████████████| 1/1 [00:02<00:00,  2.20s/it]


# Experiment 1 Variable Time

In [ ]:
# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Bird_sounds" --output_dir "exp1_new/bird_sound" --target_fs_values 11025 --input_sdrs 1 3 5 7 --time_clip 1 2 4 8 --sdr_mode 1 --pesq_mode 0 --c_win 256 > logs/bird.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Heart_sounds" --output_dir "exp1_new/heart_sound" --target_fs_values 2000 --input_sdrs 1 3 5 7 --time_clip 1 2 4 8 --sdr_mode 1 --pesq_mode 0 --c_win 512 > logs/heart.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Lung_sounds" --output_dir "exp1_new/lung_sound" --target_fs_values 4000 --input_sdrs 1 3 5 7 --time_clip 1 2 4 8 --sdr_mode 1 --pesq_mode 0 --c_win 512 > logs/lung.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Speech_sounds" --output_dir "exp1_new/speech_sound" --target_fs_values 16000 --input_sdrs 1 3 5 7 --time_clip 1 2 4 8 --sdr_mode 1 --pesq_mode 0 --c_win 1024 > logs/speech.log 2>&1 &


In [1]:
from exp_plots import plot_exp1
plot_exp1()

Plotting Experiment 1: Duration Analysis
Saved plot to /data2/AAG/Audio_Declip/saved_plots/exp1_delta_sdr.png
Saved table of values to /data2/AAG/Audio_Declip/saved_plots/exp1_delta_sdr_summary.csv
Saved plot to /data2/AAG/Audio_Declip/saved_plots/exp1_processing_time.png
Saved table of values to /data2/AAG/Audio_Declip/saved_plots/exp1_processing_time_summary.csv


# Experiment 2 Variable Sampling

In [ ]:
# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Bird_sounds" --output_dir "exp2_new/bird_sound" --target_fs_values 1378 2756 5512 11025 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --c_win 256 > logs/bird_resample.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Heart_sounds" --output_dir "exp2_new/heart_sound" --target_fs_values 250 500 1000 2000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --c_win 512 > logs/heart_resample.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Lung_sounds" --output_dir "exp2_new/lung_sound" --target_fs_values 500 1000 2000 4000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --c_win 512 > logs/lung_resample.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Speech_sounds" --output_dir "exp2_new/speech_sound" --target_fs_values 2000 4000 8000 16000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --c_win 1024 --n_files 15 > logs/speech_resample.log 2>&1 &


In [1]:
from exp_plots import plot_exp2
plot_exp2()

Plotting Experiment 2: Sampling Rate Analysis
Saved plot to /data2/AAG/Audio_Declip/saved_plots/exp2_delta_sdr.png
Saved table of values to /data2/AAG/Audio_Declip/saved_plots/exp2_delta_sdr_summary.csv
Saved plot to /data2/AAG/Audio_Declip/saved_plots/exp2_processing_time.png
Saved table of values to /data2/AAG/Audio_Declip/saved_plots/exp2_processing_time_summary.csv


# Experiment 3 Variable Blocks

In [ ]:
# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Bird_sounds" --output_dir "exp3_new/bird_sound" --target_fs_values 11025 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --c_win 110 165 254 375 573 860 1290 1940 2922 4410 --n_files 50 --exp_name timewise_len > logs/bird_blocks.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Heart_sounds" --output_dir "exp3_new/heart_sound" --target_fs_values 2000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --c_win 20 28 38 54 76 106 148 207 286 400 --n_files 50 --exp_name timewise_len > logs/heart_blocks.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Lung_sounds" --output_dir "exp3_new/lung_sound" --target_fs_values 4000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --c_win 40 60 92 136 208 312 468 704 1060 1600 --n_files 50 --exp_name timewise_len > logs/lung_blocks.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Speech_sounds" --output_dir "exp3_new/speech_sound" --target_fs_values 16000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --c_win 160 240 368 544 832 1248 1872 2816 4240 6400 --n_files 50 --exp_name timewise_len > logs/speech_blocks.log 2>&1 &


In [ ]:
from exp_plots import plot_exp3
plot_exp3()

# Dynamic ASPADE 

In [ ]:
!python evaluate.py \
--test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Bird_sounds" \
--output_dir "exp/comparison/bird_sound" \
--target_fs_values 11025 \
--input_sdrs 1 5\
--time_clip 1 \
--sdr_mode 1 \
--pesq_mode 0 \
--dynamic 0 \
--c_win 256 \
--verbose 1 \
--n_files 1 \
--stepsize 1 \
--steprate 2 \
--block_metrics 1


In [ ]:
# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Bird_sounds" --output_dir "exp/comparison/bird_sound" --target_fs_values 11025 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --dynamic 0 --c_win 256 --block_metrics 1 > logs/bird_baseline.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Bird_sounds" --output_dir "exp/comparison/bird_sound" --target_fs_values 11025 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --dynamic 1 --c_win 256 --block_metrics 1 > logs/bird_dynamic.log 2>&1 &


# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Heart_sounds" --output_dir "exp/comparison/heart_sound" --target_fs_values 2000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --dynamic 0 --c_win 512 --block_metrics 1 > logs/heart_baseline.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Heart_sounds" --output_dir "exp/comparison/heart_sound" --target_fs_values 2000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --dynamic 1 --c_win 512 --block_metrics 1 > logs/heart_dynamic.log 2>&1 &


# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Lung_sounds" --output_dir "exp/comparison/lung_sound" --target_fs_values 4000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --dynamic 0 --c_win 512 --block_metrics 1 > logs/lung_baseline.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Lung_sounds" --output_dir "exp/comparison/lung_sound" --target_fs_values 4000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --dynamic 1 --c_win 512 --block_metrics 1 > logs/lung_dynamic.log 2>&1 &


# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Speech_sounds" --output_dir "exp/comparison/speech_sound" --target_fs_values 16000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --dynamic 0 --c_win 1024 --block_metrics 1 > logs/speech_baseline.log 2>&1 &

# nohup python evaluate.py --test_audio_dir "/data2/AAG/Audio_Declip/midterm_sounds/Speech_sounds" --output_dir "exp/comparison/speech_sound" --target_fs_values 16000 --input_sdrs 1 3 5 7 --time_clip 1 --sdr_mode 1 --pesq_mode 0 --dynamic 1 --c_win 1024 --block_metrics 1 > logs/speech_dynamic.log 2>&1 &


In [1]:
from exp_plots import block_analysis
file_no=2
block_index=4
block_analysis(file_no,block_index)

Saved objective value comparison plot to /data2/AAG/Audio_Declip/exp/comparison/plots/objective_value_comparison_u2_idx4.png
Saved k value comparison plot to /data2/AAG/Audio_Declip/exp/comparison/plots/k_value_comparison_u2_idx4.png
Saved summary statistics to /data2/AAG/Audio_Declip/exp/comparison/plots/summary_statistics_u2_idx4.csv


,Sound_Type,Baseline_SDR_Mean,Baseline_SDR_Std,Baseline_SDR_Final,Dynamic_SDR_Mean,Dynamic_SDR_Std,Dynamic_SDR_Final,SDR_Improvement,Baseline_K_Mean,Dynamic_K_Mean,K_Difference,Convergence_Iterations
0,Bird,0.4702,0.3757,0.0999,0.4311,0.3144,0.0982,-0.0391,98.7494,102.9624,4.2130,266
1,Heart,3.4762,4.4551,0.0970,3.2188,3.8823,0.0927,-0.2574,225.2497,229.7890,4.5393,654
2,Lung,4.2326,6.4344,0.0921,4.0870,6.5875,0.0881,-0.1456,49.0000,47.4968,-1.5032,157
3,Speech,4.0903,5.5755,0.0993,3.8475,4.8019,0.0940,-0.2428,473.2499,480.2074,6.9575,1244


In [1]:
from exp_plots import plot_comparison
plot_comparison()

Plotting Experiment: Baseline vs Dynamic Model Comparison
Saved comparison plot to /data2/AAG/Audio_Declip/exp/comparison/plots/baseline_vs_dynamic_delta_sdr_1sec.png
Saved comparison table to /data2/AAG/Audio_Declip/exp/comparison/plots/baseline_vs_dynamic_delta_sdr_1sec.csv

=== DELTA_SDR SUMMARY (1 sec duration) ===
Bird: Baseline=5.589, Dynamic=5.591, Improvement=+0.0%
Heart: Baseline=6.824, Dynamic=6.821, Improvement=-0.0%
Lung: Baseline=6.521, Dynamic=6.375, Improvement=-2.3%
Speech: Baseline=7.326, Dynamic=7.313, Improvement=-0.2%
Saved comparison plot to /data2/AAG/Audio_Declip/exp/comparison/plots/baseline_vs_dynamic_cycles_1sec.png
Saved comparison table to /data2/AAG/Audio_Declip/exp/comparison/plots/baseline_vs_dynamic_cycles_1sec.csv

=== CYCLES SUMMARY (1 sec duration) ===
Bird: Baseline=63765.923, Dynamic=50212.027, Improvement=-21.3%
Heart: Baseline=14083.548, Dynamic=10868.413, Improvement=-22.8%
Lung: Baseline=12879.875, Dynamic=11124.883, Improvement=-13.6%
Speech: B

# ML-ASPADE

In [ ]:
!python evaluate.py --model_path "saved_models/final/complex_dft_unet_final.pth" \
    --test_audio_dir "custom_sound" \
    --output_dir "custom_sound_output" \
    --target_fs_values 16000 \
    --clipping_thresholds 0.2 \
    --time_clip 1 \
    --factor 0.8 \
    --eval_mode 1 \
    --dynamic 0 \
    --save 0 \
    --delta 300 \
    --c_win 500 \
    --r_mode 1 \
    --verbose 1

# LibriData Evaluation

In [2]:
import pandas as pd

# File paths
baseline_file = "/data2/AAG/Audio_Declip/custom_output/evaluation_results_baseline_model.xlsx"
dynamic_file = "/data2/AAG/Audio_Declip/custom_output/evaluation_results_dynamic_model.xlsx"
ml_file = "/data2/AAG/Audio_Declip/custom_output/evaluation_results_ml_model.xlsx"

# Load data
df_baseline = pd.read_excel(baseline_file)
df_dynamic = pd.read_excel(dynamic_file)
df_ml = pd.read_excel(ml_file)

# Combine in a dict
models = {
    "Baseline": df_baseline,
    "Dynamic": df_dynamic,
    "ML": df_ml
}

# Metrics to summarize
metrics = ['delta_sdr', 'delta_pesq', 'processing_time', 'cycles']
thresholds = [0.1, 0.3]

# Print header
print(f"{'Threshold':<10} {'Model':<10} {'Delta SDR':<20} {'Delta PESQ':<20} {'Time (s)':<20} {'Cycles':<20}")

# Loop by threshold first
for threshold in thresholds:
    for model_name, df in models.items():
        group = df[df['threshold'] == threshold]
        values = []
        for metric in metrics:
            mean = group[metric].mean()
            std = group[metric].std()
            values.append(f"{mean:.2f} ({std:.2f})")
        print(f"{threshold:<10} {model_name:<10} {values[0]:<20} {values[1]:<20} {values[2]:<20} {values[3]:<20}")


Threshold  Model      Delta SDR            Delta PESQ           Time (s)             Cycles              
0.1        Baseline   18.33 (2.17)         1.01 (0.42)          4.76 (1.04)          36733.32 (8044.73)  
0.1        Dynamic    18.29 (2.10)         0.97 (0.43)          4.37 (0.92)          26530.55 (5652.93)  
0.1        ML         17.96 (2.20)         1.05 (0.40)          4.73 (0.75)          20777.75 (4455.39)  
0.3        Baseline   20.69 (4.07)         1.49 (0.55)          5.71 (1.02)          45176.12 (8067.86)  
0.3        Dynamic    20.66 (4.02)         1.48 (0.55)          5.27 (0.93)          32789.71 (5745.81)  
0.3        ML         21.03 (3.85)         1.53 (0.53)          5.32 (0.72)          25391.88 (4556.79)  


In [4]:
!python plot_loss_history.py --history_file /data2/AAG/Audio_Declip/saved_models/loss_history.json

Plots saved to loss_plots


# Extras